# 15 P1 Project

* ist1102903 Ana Alfaiate (33%)
  
* ist1102902 Maria Inês Trigueiro (33%)
  
* ist102881 Raquel Coelho (33%)

Prof. Flávio Martins (and Prof. João Marques)

Lab Shift number: PB03

## PART I – E-R Model

#### 1. Proposed database design


The E-R model describing our proposed database design is represented below.



Every design decision that can be captured in the E-R model should be represented in the diagram. Do not forget to include additional integrity constraints not captured in the E-R model. You may provide notes explaining the rationale behind non-trivial design decisions.

### E-R Model

![E-R Model](../images/15-p1-er-model.png "E-R Model")

## PART II – Relational Model

### Database Schema

#### 1. Create the tables and integrity constraints corresponding to the relational database schema obtained.

% Do not forget to include additional integrity constraints not captured in the relational model.

%Please use database `db`, which should already have been created. If not, follow the instructions in Lab01 to create it.

### Create the Database vetDB


- Connect to PostgreSQL using the `psql` command-line interface.

```bash
psql -h postgres -U postgres ⮐
```

- Enter the password for the user `postgres`.

   `postgres`↵

- Create a new unprivileged user `vetDB`.

   ```sql
   CREATE USER vetDB WITH PASSWORD 'vetDB'; ⮐
   ```

- Create database `vetDB` and set user `vetDB` as owner of the database.

    ```sql
    CREATE DATABASE vetDB
    WITH
    OWNER = vetDB
    ENCODING = 'UTF8'; ⮐
    ```

- Grant all privileges on the database `vetDB` to the user `vetDB`.

```sql
GRANT ALL ON DATABASE vetDB TO vetDB; ⮐
```

- Connect to PostgreSQL using the `psql` command-line interface.

```bash
psql -h postgres -U vetDB ⮐
```

In [ ]:
%load_ext sql
%%sql postgresql+psycopg://db:db@postgres/db

In [1]:
%%sql

DROP TABLE IF EXISTS Measurement;
DROP TABLE IF EXISTS Test;
DROP TABLE IF EXISTS Other_Procedures;
DROP TABLE IF EXISTS Surgical;
DROP TABLE IF EXISTS Radiography;
DROP TABLE IF EXISTS performs;
DROP TABLE IF EXISTS Procedure;
DROP TABLE IF EXISTS Medication;
DROP TABLE IF EXISTS Prescription;
DROP TABLE IF EXISTS associated;
DROP TABLE IF EXISTS Diagnostic_code;
DROP TABLE IF EXISTS assists;
DROP TABLE IF EXISTS Veterinary_Visit;
DROP TABLE IF EXISTS Assistant;
DROP TABLE IF EXISTS Client;
DROP TABLE IF EXISTS Doctor;
DROP TABLE IF EXISTS Cats_or_Dogs;
DROP TABLE IF EXISTS Other_Animals;
DROP TABLE IF EXISTS Animal;


CREATE TABLE Client (
    tin NUMERIC(9),
    name VARCHAR(80),
    address_line_1 VARCHAR(255),
    address_line_2 VARCHAR(255),
    postcode VARCHAR(12),
    city VARCHAR(30),
    phone_nr VARCHAR(15),
    PRIMARY KEY (tin),
    UNIQUE (phone_nr)
);

CREATE TABLE Assistant (
    name VARCHAR(80),
    address_line_1 VARCHAR(255),
    address_line_2 VARCHAR(255),
    postcode VARCHAR(12),
    city VARCHAR(30),
    phone_nr VARCHAR(15),
    PRIMARY KEY (phone_nr)
);

CREATE TABLE Doctor (
    omv NUMERIC(9),
    name VARCHAR(80),
    address_line_1 VARCHAR(255),
    address_line_2 VARCHAR(255),
    postcode VARCHAR(12),
    city VARCHAR(30),
    phone_nr VARCHAR(15),
    specialization VARCHAR (80),
    biog_description TEXT,
    PRIMARY KEY (omv),
    UNIQUE (phone_nr)
);

CREATE TABLE Animal (
    name VARCHAR(80),
    owner VARCHAR(80),
    birthdate DATE,
    color VARCHAR(30),
    gender VARCHAR(9),
    weight_g NUMERIC(9,3),
    PRIMARY KEY (name, owner)
    -- No animal can exist at the same time in both the table 'Cats_or_Dogs' or in the table 'Other_Animals'
);

CREATE TABLE Cats_or_Dogs (
    name VARCHAR(80),
    owner VARCHAR(80),
    breed VARCHAR(80),
    PRIMARY KEY (name, owner),
    FOREIGN KEY (name, owner) REFERENCES Animal(name, owner)
);

CREATE TABLE Other_Animals (
    name VARCHAR(80),
    owner VARCHAR(80),
    spc VARCHAR(80),
    PRIMARY KEY (name, owner),
    FOREIGN KEY (name, owner) REFERENCES Animal(name, owner)
);

CREATE TABLE Veterinary_Visit (
    name VARCHAR(80),
    owner VARCHAR(80),
    date TIMESTAMP,
    S TEXT,
    O TEXT,
    A TEXT,
    P TEXT,
    omv NUMERIC(9),
    tin NUMERIC(9),
    PRIMARY KEY (name, owner, date),
    FOREIGN KEY (name, owner) REFERENCES Animal(name, owner)
    FOREIGN KEY (omv) REFERENCES Doctor(omv)
    FOREIGN KEY (tin) REFERENCES Client(tin)
);

CREATE TABLE assists (
    name VARCHAR(80),
    owner VARCHAR(80),
    date TIMESTAMP,
    phone_nr VARCHAR(15),
    PRIMARY KEY (name, owner, date, phone_nr),
    FOREIGN KEY (name, owner, date) REFERENCES Veterinary_Visit(name, owner, date)
    FOREIGN KEY (phone_nr) REFERENCES Assistant(phone_nr)
);

CREATE TABLE Diagnostic_code (
    code VARCHAR(20), --confirmar tipo
    description TEXT,
    PRIMARY KEY (code),
);

CREATE TABLE associated (
    name VARCHAR(80),
    owner VARCHAR(80),
    date TIMESTAMP,
    code VARCHAR(20), --confirmar tipo
    PRIMARY KEY (name, owner, date, code),
    FOREIGN KEY (name, owner, date) REFERENCES Veterinary_Visit(name, owner, date)
    FOREIGN KEY (code) REFERENCES Diagnostic_code(code)
);

CREATE TABLE Prescription (
    prescription_id INTEGER,
    name VARCHAR(80),
    owner VARCHAR(80),
    date TIMESTAMP,
    code VARCHAR(20), --confirmar tipo
    PRIMARY KEY (prescription_id),
    FOREIGN KEY (name, owner, date, code) REFERENCES associated(name, owner, date, code)
    -- Every prescription must appear in the table 'medication' 
);

CREATE TABLE Medication (
    prescription_id INTEGER,
    name VARCHAR(80),
    lab VARCHAR(80),
    dosage VARCHAR(80),
    PRIMARY KEY (prescription_id, name, lab, dosage),
    FOREIGN KEY (prescription_id) REFERENCES Prescription(prescription_id)
);

CREATE TABLE Procedure (
    description VARCHAR(255),
    report TEXT,
    name VARCHAR(80),
    owner VARCHAR(80),
    date TIMESTAMP,
    PRIMARY KEY (description),
    FOREIGN KEY (name, owner, date) REFERENCES Veterinary_Visit(name, owner, date)
    -- Every procedure must exist in either the table  'Test' or in the table 'Other_Procedures', or in the table 'Surgical' or in the table 'Radiography'
    -- No procedure can exist simultaneously in more than one of the following tables: 'Test', 'Other_Procedures', 'Surgical', or 'Radiography'
);

CREATE TABLE performs (
    phone_nr VARCHAR(15),
    description VARCHAR(255),
    PRIMARY KEY (phone_nr, description),
    FOREIGN KEY (phone_nr) REFERENCES Assistant(phone_nr),
    FOREIGN KEY (description) REFERENCES Procedure(description)
);

CREATE TABLE Radiography (
    description VARCHAR(255),
    img_path VARCHAR(255),
    PRIMARY KEY (description),
    FOREIGN KEY (description) REFERENCES Procedure(description)
);

CREATE TABLE Surgical (
    description VARCHAR(255),
    PRIMARY KEY (description),
    FOREIGN KEY (description) REFERENCES Procedure(description)
);

CREATE TABLE Other_Procedures (
    description VARCHAR(255),
    PRIMARY KEY (description),
    FOREIGN KEY (description) REFERENCES Procedure(description)
);

CREATE TABLE Test (
    description VARCHAR(255),
    PRIMARY KEY (description),
    FOREIGN KEY (description) REFERENCES Procedure(description)
);

CREATE TABLE Measurement (
    description VARCHAR(255),
    indicator VARCHAR(30),
    ref_range VARCHAR(30),
    range_description VARCHAR(255),
    values REAL,
    units VARCHAR(30),
    PRIMARY KEY (description),
    FOREIGN KEY (description) REFERENCES Procedure(description)
);


U
s
a
g
e
E
r
r
o
r
:
 
C
e
l
l
 
m
a
g
i
c
 
`
%
%
s
q
l
`
 
n
o
t
 
f
o
u
n
d
.

